In [1]:
import numpy as np
from  sklearn.datasets import make_circles
import matplotlib.pyplot as plt
import pandas as pd
from   itertools import combinations
from scipy.stats import multivariate_normal
import random

In [2]:
def normalize(X):
    mu = np.mean(X,axis=0)
    X_norm = X - mu
    sigma = np.std(X_norm,axis=0)
    X_norm = X_norm/sigma
    return X_norm

class Maximum_subspace_representation():
    def __init__(self,kernel='rbf', sigma=5.0, degree=3, coef=1):
        self.kernel = kernel
        self.sigma = sigma
        self.degree = degree
        self.coef = coef
    
    def gaussian_kernel(self,x1,x2,sigma):
        return np.sum(np.exp(-np.power(x1-x2,2)/(2*(sigma**2))))
    
    def polynomial_kernel(self,x1,x2,d=3,r=1.0):
        return (np.dot(x1,x2.T)+r)**d
    
    def linear_kernel(self,x1,x2):
        return np.dot(x1,x2.T)
    
    def kernel_matrix(self,X1,X2):
        m,n = X1.shape[0],X2.shape[0]
        K = np.zeros((m,n))
        
        if self.kernel == 'rbf':
            for i in range(m):
                for j in range(n):
                    K[i,j] = self.gaussian_kernel(X1[i],X2[j],self.sigma)
        elif self.kernel == 'poly':
            for i in range(m):
                for j in range(n):
                    K[i,j] = self.polynomial_kernel(X1[i],X2[j],self.degree,self.coef)
        elif self.kernel == 'linear':
            for i in range(m):
                for j in range(n):
                    K[i,j] = self.linear_kernel(X1[i],X2[j])
        
        K=np.power(K,2)
        return K
    
    def center_kernel_matrix(self,K):
        m = K.shape[0]
        N = np.ones((m,m))/m
        K_centered = K - np.dot(N,K) - np.dot(K,N) + np.dot(np.dot(N,K),N)
        return K_centered
    
    def get_eig(self,K):
        eigvals,eigvecs = np.linalg.eig(K)
        idx = eigvals.argsort()[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:,idx]
        return eigvals,eigvecs
    
    def transform(self,X):
        K = self.kernel_matrix(X,X)
        K_centered = self.center_kernel_matrix(K)
        eigvals,eigvecs = self.get_eig(K_centered)
        self.eigvals = eigvals
        self.eigvecs = eigvecs
        X_reduced = np.dot(K_centered,eigvecs[:,:self.n_components])
        return X_reduced
        
    def fit(self,X,n_components):
        self.n_components = n_components
        self.X = X
        X_norm = normalize(X)
        self.X_reduced = self.transform(X_norm)

In [3]:
def simulated_data(u,s,p=0):
    m1,m2,m3,m4 = np.array([-u+p, u]), np.array([u+p, u]),np.array([-u+p, -u]), np.array([u+p, -u])
    s1,s2,s3,s4= np.array([[s, 0], [0, s]]),np.array([[s, 0], [0, s]]),np.array([[s, 0], [0, s]]),np.array([[s, 0], [0, s]])
    N1 = np.random.multivariate_normal(m1, s1, size=1000)
    N2 = np.random.multivariate_normal(m2, s2, size=1000)
    N3 = np.random.multivariate_normal(m3, s3, size=1000)
    N4 = np.random.multivariate_normal(m4, s4, size=1000)
    return np.vstack([N1,N2,N3,N4])



data1=simulated_data(2.0,0.05)
data2=simulated_data(1.8,0.05)
data3=simulated_data(1.6,0.05)
data4=simulated_data(1.4,0.05)
data5=simulated_data(1.2,0.05)
data6=simulated_data(1.0,0.05)

'''
data1=simulated_data(2.,0.05,0)
data2=simulated_data(1.8,0.1,0)
data3=simulated_data(1.6,0.2,0)
data4=simulated_data(1.4,0.4,0)
data5=simulated_data(1.2,0.6,0)
data6=simulated_data(1.0,0.8,0)
'''
'''
data1=simulated_data(2.0,0.05,0)
data2=simulated_data(2.0,0.05,0.5)
data3=simulated_data(2.0,0.05,1)
data4=simulated_data(2.0,0.05,1.5)
data5=simulated_data(2.0,0.05,2.)
data6=simulated_data(2.0,0.05,2.5)
'''

'\ndata1=simulated_data(2.0,0.05,0)\ndata2=simulated_data(2.0,0.05,0.5)\ndata3=simulated_data(2.0,0.05,1)\ndata4=simulated_data(2.0,0.05,1.5)\ndata5=simulated_data(2.0,0.05,2.)\ndata6=simulated_data(2.0,0.05,2.5)\n'

In [4]:
def RSD(Feature_s, Feature_t):
    
    Feature_s=np.array(Feature_s, dtype="float32")
    Feature_t=np.array(Feature_t, dtype="float32")
    
    u_s, s_s, v_s = np.linalg.svd(Feature_s.T,full_matrices=False)
    u_t, s_t, v_t = np.linalg.svd(Feature_t.T,full_matrices=False)
    
    p_s1, cospa1, p_t1 = np.linalg.svd(np.matmul(u_s.T, u_t),full_matrices=False)
    p_s2, cospa2, p_t2 = np.linalg.svd(np.matmul(v_s, v_t.T),full_matrices=False)
    
    return np.sum(cospa1)/cospa1.shape[0]+np.sum(cospa2)/cospa2.shape[0]

def dim_selection(a,threshold):
    flag=True
    for i in range(len(a)):
        if np.sum(a[:i+1])/np.sum(a)>=threshold and flag:
            dim_opt=i+1
            flag=False
    return dim_opt

def maixmum_subspace(data_list,sample_num,thre):
    shuffle=np.random.permutation(data_list[0].shape[0])
    initial_dim=10
    name_list=[]
    reduced_list=[]

    for i in range(len(data_list)):
        if i==0:
            initial_data=data_list[i][shuffle][:sample_num]
            all_data=initial_data
        else:
            all_data=np.vstack((all_data,data_list[i][shuffle][:sample_num]))
    
    msr = Maximum_subspace_representation(kernel='rbf',sigma=5)
    msr.fit(all_data,initial_dim)
    
    reduce_dim=dim_selection(msr.eigvals,thre)
    msr.fit(all_data,reduce_dim)

    for i in range(len(data_list)):
        name_list.append('data'+str(i+1))
        reduced_list.append(msr.X_reduced[i*sample_num:(i+1)*sample_num])
    
    data_combination=list(combinations(reduced_list,2))[:5]
    name_combination=list(combinations(name_list,2))[:5]
    data_angle=[]
    data_trans=[]
    
    for i in data_combination:
        data_angle.append(RSD(i[0],i[1]))
        
    for i in (data_angle):
        data_trans.append((i-np.min((data_angle)))/(np.max((data_angle))-np.min((data_angle))))
    
    print(name_combination,data_angle,data_trans)

In [5]:
list_data = [data1,data2,data3,data4,data5,data6]
maixmum_subspace(data_list=list_data,sample_num=100,thre=0.8)

[('data1', 'data2'), ('data1', 'data3'), ('data1', 'data4'), ('data1', 'data5'), ('data1', 'data6')] [1.9874494075775146, 1.9869426488876343, 1.9793809652328491, 1.973952829837799, 1.9632560014724731] [1.0, 0.9790538509674844, 0.6665024217906962, 0.44213817264435895, 0.0]


C:\TEM\ipykernel_12832\1095489317.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  Feature_s=np.array(Feature_s, dtype="float32")
C:\TEM\ipykernel_12832\1095489317.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  Feature_t=np.array(Feature_t, dtype="float32")
